In [1]:
import pandas as pd
import numpy as np
import copy
import csv
import itertools

from tqdm import tqdm
from Bio import SeqIO, SearchIO 

In [2]:
rodeo = pd.read_csv('/home/niagara/Storage/Marfa/M_Zakirova/projects/mycofactocin/mftb/hmm_parsed_domain/main_co_occur_rodeo_psi.csv', sep = ",")
rodeo = rodeo.loc[:,['Query','Protein_acc']]
rodeo

,Query,Protein_acc
0,WP_145853129.1,WP_187365872.1
1,WP_145853129.1,WP_187365873.1
2,WP_145853129.1,WP_145853125.1
3,WP_145853129.1,WP_145853126.1
4,WP_145853129.1,WP_145853127.1
...,...,...
11152,WP_015759610.1,WP_015759614.1
11153,WP_015759610.1,WP_015759615.1
11154,WP_015759610.1,WP_015759616.1
11155,WP_015759610.1,WP_015759617.1


__Посчитать, сколько раз во всех наших геномных регионах/оперонах встретилась доменная архитектура  "домен X - домен Y - ... - домен N"__

In [75]:
file = '/home/niagara/Storage/Marfa/M_Zakirova/projects/mycofactocin/mftb/hmm_parsed_domain/parsed_hmm_per_dom_tbl.txt'
res_domains = dict()
x = 10
count = 0
for res in SearchIO.parse(file, 'hmmscan3-domtab'):
        res_domains[res.id] = {}
        for hit in res.hits:
            if hit.id[:4] == 'TIGR':
                    call = hit.description.split(":")[0]
            else:
                    call = hit.id
            for hsp in hit.hsps:
                overlap = False
                if res_domains[res.id].keys(): #not empty
                    names = copy.deepcopy(list(res_domains[res.id].keys()))
                    for name in names: #проверка на перекрытие с раннее добавленными генами 
                        for q_range in res_domains[res.id][name]:
                            if min(hsp.env_end, q_range[1]) - max(hsp.env_start, q_range[0]) > x:
                                overlap = True
                                break
                        if overlap:
                            break
                    if not overlap: 
                        if call not in res_domains[res.id]:
                            res_domains[res.id][call] = [(hsp.env_start, hsp.env_end)]
                            
                        if (hsp.env_start, hsp.env_end) not in res_domains[res.id][call]:
                                        #для случая мультидоменного белка с одинаковыми доменами: 
                                        #один и тот же белок выравнивается на несколько мест
                            res_domains[res.id][call].append((hsp.env_start, hsp.env_end))
                       
                else:
                    res_domains[res.id][call] = [(hsp.env_start, hsp.env_end)]
                    

In [76]:
domains_names = {}
for key, value in res_domains.items():
    domains_names[key] = list(value.keys())

domains_names = pd.DataFrame({'domains': domains_names.values()}, index =res_domains.keys())
domains_names

,domains
WP_011566059.1,[MnhB]
WP_011566060.1,[2a6301s01]
WP_011566061.1,"[mycofact_glyco, Methyltransf_23, Methyltransf..."
WP_011566062.1,[Creatininase]
WP_011566063.1,[actino_HemFlav]
...,...
WP_160900598.1,"[DoxX_3, DoxX]"
WP_160901386.1,[DNA_alkylation]
WP_160900597.1,"[Pyr_redox_2, Reductase_C]"
WP_160900596.1,"[FAM70, DUF3040]"


In [113]:
domains_region = {k:[] for k in rodeo['Query'].unique()}

for query in tqdm(domains_region.keys()):
    temp = rodeo[rodeo['Query'] == query]
    for prot_id in temp['Protein_acc']:
        if prot_id in domains_names.index:
            for domains in domains_names.loc[prot_id]:
                domains_region[query] += domains
dom_region = pd.DataFrame({'domains': domains_region.values()} , index = domains_region.keys())
dom_region

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 681/681 [00:03<00:00, 184.57it/s]


,domains
WP_145853129.1,"[STAS_2, TetR_C_1, TetR_N, CSD, pcrA, OsmC, my..."
WP_056154608.1,"[efflux_EmrB, DUF6286, mycofact_glyco, SDR_sub..."
WP_111258449.1,"[PemK_toxin, HicB, Arc, WYL, HTH_11, HTH_PafC,..."
WP_212516844.1,"[3oxo_ACP_reduc, kduD, pyruv_carbox, FCD, GntR..."
WP_040718660.1,"[NDMA_methanol, AAA_5, Sigma54_activ_2, 26Sp45..."
...,...
WP_212844265.1,"[UreD, salicylate_mono, TetR_C_31, Laminin_G_3..."
WP_050670291.1,"[2TM, DUF779, Aldedh, actino_HemFlav, mycofact..."
WP_122191250.1,"[Aldedh, PA_CoA_ligase, Rxyl_3153, adh_short_C..."
WP_011208766.1,"[VWA_CoxE, Methyltransf_25, meth_Rta_06860, PA..."


In [124]:
array = list(str(val) for val in domains_region.values())
u, counts = np.unique(array, return_counts=True)
domains_count_regions = pd.DataFrame({'domains': u, 'counts': counts})
domains_count_regions = domains_count_regions.sort_values(by = ['counts'], ascending=False).reset_index(drop=True)
domains_count_regions

,domains,counts
0,"['EutC', 'EutB', '2A0305', 'Rv0697', 'mycofact...",2
1,"['COesterase', 'CYP450_TxtE', 'TetR_N', 'TetR_...",2
2,"['TetR_N', 'TetR_C_11', 'efflux_EmrB', 'Asp_Gl...",2
3,"['Usp', '2A0306', 'Orn_aminotrans', 'DDAH_euka...",2
4,"['2-Hacid_dh_C', 'AdoHcyase_NAD', 'FecCD', 'Fe...",1
...,...,...
672,"['Fer4', 'Fer4_13', 'Sig70_famx2', 'Aldedh', '...",1
673,"['Fer4', 'flavo_MJ0208', 'DUF2249', 'CysN', 'A...",1
674,"['Fer4_15', 'APH', 'mycofact_TetR', 'HTH_7', '...",1
675,"['Fer4_2', 'Fer4', 'zf-C2H2_11', 'GG-red-SF', ...",1


In [129]:
#domains_count_regions.to_csv('/home/niagara/Storage/Marfa/M_Zakirova/projects/mycofactocin/mftb/hmm_parsed_domain/counts/domains_arch_regions.csv')

__Для оперонов:__

In [104]:
operon_prot_id = pd.read_csv('/home/niagara/Storage/Marfa/M_Zakirova/projects/mycofactocin/mftb/hmm_parsed_domain/counts/operon_100_protIDs_rodeo.csv', sep = ",", index_col = 0)
operon_prot_id

,Query,Protein_acc
1,WP_145853129.1,WP_246115648.1
2,WP_145853129.1,WP_246115687.1
3,WP_145853129.1,WP_145853128.1
4,WP_145853129.1,WP_145853129.1
5,WP_145853129.1,WP_145853130.1
...,...,...
5243,WP_015759610.1,WP_015759614.1
5244,WP_015759610.1,WP_015759615.1
5245,WP_015759610.1,WP_015759616.1
5246,WP_015759610.1,WP_015759617.1


In [115]:
domains_operon = {k:[] for k in operon_prot_id['Query'].unique()}

for query in tqdm(domains_operon.keys()):
    temp = operon_prot_id[operon_prot_id['Query'] == query]
    for prot_id in temp['Protein_acc']:
        if prot_id in domains_names.index:
            for domains in domains_names.loc[prot_id]:
                domains_operon[query] += domains
dom_operon = pd.DataFrame({'domains': domains_operon.values()} , index = domains_operon.keys())
dom_operon

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 675/675 [00:02<00:00, 330.61it/s]


,domains
WP_145853129.1,"[OsmC, mycofact_glyco, mycofact_rSAM, Mycofact..."
WP_056154608.1,"[actino_HemFlav, mycofact_rSAM, Mycofactocin_R..."
WP_111258449.1,"[mycofactocin, Mycofactocin_RRE, mycofact_rSAM..."
WP_212516844.1,"[FCD, GntR, P450_rel_GT_act, mycofactocin, Myc..."
WP_040718660.1,"[actino_HemFlav, mycofact_rSAM, Mycofactocin_R..."
...,...
WP_212844265.1,"[mycofactocin, Mycofactocin_RRE, mycofact_rSAM..."
WP_050670291.1,"[SDR_subfam_2, mycofact_creat, mycofact_rSAM, ..."
WP_122191250.1,"[mycofactocin, Mycofactocin_RRE, mycofact_rSAM..."
WP_011208766.1,"[actino_HemFlav, mycofact_rSAM, Mycofactocin_R..."


In [116]:
array_op = list(str(val) for val in domains_operon.values())
u_op, counts_op = np.unique(array_op, return_counts=True)
domains_count_operon = pd.DataFrame({'domains': u_op, 'counts': counts_op})
domains_count_operon = domains_count_operon.sort_values(by = ['counts'], ascending=False).reset_index(drop=True)
domains_count_operon

,domains,counts
0,"['mycofactocin', 'Mycofactocin_RRE']",6
1,"['actino_HemFlav', 'mycofact_rSAM', 'Mycofacto...",4
2,"['mycofact_creat', 'actino_HemFlav', 'mycofact...",4
3,"['mycofact_creat', 'actino_HemFlav', 'mycofact...",4
4,"['SDR_subfam_1', 'actino_HemFlav', 'mycofact_r...",3
...,...,...
623,"['SDR_subfam_1', 'SDR_subfam_1', 'SDR_subfam_1...",1
624,"['SDR_subfam_1', 'SDR_subfam_1', 'SDR_subfam_1...",1
625,"['SDR_subfam_1', 'SDR_subfam_1', 'SDR_subfam_1...",1
626,"['SDR_subfam_1', 'SDR_subfam_1', 'SDR_subfam_1...",1


In [131]:
#domains_count_operon.to_csv('/home/niagara/Storage/Marfa/M_Zakirova/projects/mycofactocin/mftb/hmm_parsed_domain/counts/domains_arch_operon.csv')

---

__Посчитать, сколько раз во всех наших геномных регионах/оперонах одновременно присутствуют домен X и домен Y__

In [128]:
domain_counts_regions = dict()
for ind in tqdm(dom_region.index):
    buffer = dom_region.loc[ind,:]
    pair_order_list = itertools.combinations(buffer['domains'], 2)
    for pair in pair_order_list:
        if pair not in domain_counts_regions and pair[::-1] not in domain_counts_regions:
            domain_counts_regions[pair] = 1
        elif pair in domain_counts_regions:
            domain_counts_regions[pair] += 1
        elif pair[::-1] in domain_counts_regions:
            domain_counts_regions[pair[::-1]] += 1  

domain_counts_regions = pd.DataFrame({'prot_1':[key[0] for key in domain_counts_regions.keys()],'prot_2':[key[1] for key in domain_counts_regions.keys()], 'counts':domain_counts_regions.values()})
domain_counts_regions = domain_counts_regions.sort_values(by = ['counts'], ascending=False).reset_index(drop=True)
domain_counts_regions = domain_counts_regions[domain_counts_regions['counts'] >= 3]
domain_counts_regions


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 681/681 [00:00<00:00, 1507.65it/s]


,prot_1,prot_2,counts
0,Mycofactocin_RRE,mycofactocin,654
1,mycofact_rSAM,mycofactocin,646
2,mycofact_rSAM,Mycofactocin_RRE,642
3,actino_HemFlav,mycofact_rSAM,493
4,mycofactocin,mycofact_TetR,492
...,...,...,...
7272,Bac_luciferase,PP-binding,3
7273,PP-binding,menH_SHCHC,3
7274,DHPS,mycofact_TetR,3
7275,AAA_5,Pribosyltran,3


In [ ]:
#domain_counts_regions.to_csv('/home/niagara/Storage/Marfa/M_Zakirova/projects/mycofactocin/mftb/hmm_parsed_domain/counts/domains_pair_regions.csv')

In [127]:
domain_counts_operons = dict()
for ind in tqdm(dom_operon.index):
    buffer = dom_operon.loc[ind,:]
    pair_order_list = itertools.combinations(buffer['domains'], 2)#все пары генов для данного query
    for pair in pair_order_list:
        if pair not in domain_counts_operons and pair[::-1] not in domain_counts_operons:
            domain_counts_operons[pair] = 1
        elif pair in domain_counts_operons:
            domain_counts_operons[pair] += 1
        elif pair[::-1] in domain_counts_operons:
            domain_counts_operons[pair[::-1]] += 1  #чтобы избавиться от ориентированных пар

domain_counts_operons = pd.DataFrame({'prot_1':[key[0] for key in domain_counts_operons.keys()],'prot_2':[key[1] for key in domain_counts_operons.keys()], 'counts':domain_counts_operons.values()})
domain_counts_operons = domain_counts_operons.sort_values(by = ['counts'], ascending=False).reset_index(drop=True)
domain_counts_operons = domain_counts_operons[domain_counts_operons['counts'] >= 3]
domain_counts_operons


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 675/675 [00:00<00:00, 3265.83it/s]


,prot_1,prot_2,counts
0,mycofact_rSAM,Mycofactocin_RRE,607
1,Mycofactocin_RRE,mycofactocin,576
2,mycofact_rSAM,mycofactocin,551
3,actino_HemFlav,mycofact_rSAM,354
4,actino_HemFlav,Mycofactocin_RRE,347
...,...,...,...
1351,mycofact_OYE_2,zf-C2H2_11,3
1352,aprB,GG-red-SF,3
1353,SDR_subfam_1,fumC_II,3
1354,aprB,ETF_alpha,3


In [132]:
#domain_counts_operons.to_csv('/home/niagara/Storage/Marfa/M_Zakirova/projects/mycofactocin/mftb/hmm_parsed_domain/counts/domains_pair_operons.csv')